# Large Scale Data Mining 1st assignment
Francisco Marques 97639

#### Initialize Spark session and load file as RDD

In [9]:
# issues with SPARK_HOME path, the next 2 lines solve the issue for MY computer
import os
os.environ['SPARK_HOME'] = r"C:\PySpark\spark-3.3.2-bin-hadoop3"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

conf = SparkConf().setMaster("local[*]").setAppName("abc").set("spark.driver.memory", "6G").set("spark.executor.memory", "6G")
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=abc, master=local[*]) created by __init__ at C:\Users\franc\AppData\Local\Temp\ipykernel_2016\2263445683.py:11 

In [12]:
support_threshold = 1000
k = [2, 3]
inputFile = 'conditions.csv'
file = sc.textFile(inputFile).sample(False, 0.01).repartition(1000) # load file as RDD

Map the RDD to split the lines and put all of the items in just one array

It only keeps the columns PATIENT and CODE.

In [13]:
# lines in one array
full_log = (file.map(lambda line: line.split(','))
        .flatMap(lambda item: (item[2], item[-2])) # keeps PATIENT and CODE
        )

full_log.take(10)

['d35817fc-4f54-4bba-8747-8e645d52d5f5',
 '444814009',
 'fc6a54e0-7d86-4827-9fb2-3fcf7ba7d01d',
 '65363002',
 '3beb4761-c637-43ef-8587-1d2b287f31fe',
 '444814009',
 '91d867a8-452c-4873-a6ca-2942206332a8',
 '10509002',
 'dee37c62-fc5e-4d93-b87a-b7acc62b8878',
 '68496003']

Create new RDD with tuples (PATIENT, CODE)

In [14]:
patientRdd = (file.map(lambda line: line.split(','))
            .map(lambda item: (item[2], item[-2]))
            )

patientRdd.take(10)

[('d35817fc-4f54-4bba-8747-8e645d52d5f5', '444814009'),
 ('fc6a54e0-7d86-4827-9fb2-3fcf7ba7d01d', '65363002'),
 ('3beb4761-c637-43ef-8587-1d2b287f31fe', '444814009'),
 ('91d867a8-452c-4873-a6ca-2942206332a8', '10509002'),
 ('dee37c62-fc5e-4d93-b87a-b7acc62b8878', '68496003'),
 ('6c5ad5ae-4bba-49ad-b706-9b1ab83e6e68', '43878008'),
 ('7e5dcd93-1035-450c-96a3-1aff8a6da3a6', '195662009'),
 ('59f9b891-0222-4f56-96d1-9fc051d3a8c7', '15777000'),
 ('1a110c18-585d-48a9-9945-13fc2d0fa24f', '15777000'),
 ('c32840a6-eefa-45be-9d65-f75f0ca538fd', '15777000')]

Removing duplicates as a patient can be diagnosed multiple times for the same disease, but not have several of the same diseases.

In [15]:
unique_logs = patientRdd.distinct() 
unique_logs.take(10)

Group the diseases for each patient

In [ ]:
groupedRdd = unique_logs.groupByKey().mapValues(tuple)

groupedRdd.take(10)

[('7193f833-fbdb-464a-9cb2-f05c2af27b68', ('428251008',)),
 ('1e1656d8-2307-4204-b853-6118233aa810', ('428251008',)),
 ('aa12b6fb-ffa4-4146-ad28-3ee28fe61193', ('428251008',)),
 ('50c364d4-186a-486c-a869-3c2c2e252924', ('428251008', '72892002')),
 ('d258f2a8-0b63-4574-9640-fa86f9ed3efd', ('428251008',)),
 ('f6502af8-7224-4c42-95cf-49500936c5d6', ('428251008',)),
 ('72ddc54f-e898-496c-848f-ddeb6a23775e', ('428251008',)),
 ('64a9d8e3-558f-4d68-bd48-652ced1ea6c4', ('428251008',)),
 ('f3e5cad3-2831-4ba7-8e10-801bc4c3a41e', ('428251008', '74400008')),
 ('3c4fc8ef-66f1-4e07-b95e-9e4c4cebe948', ('428251008',))]

Map a new RDD (item, 1), and count and filter each item in the RDD with ungrouped items. Then reassign the RDD without the counts.

In [ ]:
freqRdd = (full_log.map(lambda item: (item, 1))
        .reduceByKey(lambda v1, v2: v1 + v2)
        .filter(lambda item: item[1] >= support_threshold)
        )

freqRdd.top(10) # 10 most frequent items, for k = 1

[('72892002', 5138),
 ('65363002', 1859),
 ('59621000', 3150),
 ('55822004', 1307),
 ('444814009', 12507),
 ('44465007', 1315),
 ('43878008', 1701),
 ('40055000', 2517),
 ('271737000', 3551),
 ('195662009', 6907)]

Transform the frequent itemset RDD back to keys only

In [ ]:
freqRdd = freqRdd.map(lambda item: item[0])
freqRdd.take(5)

['44465007', '19169002', '55822004', '15777000', '10509002']

Need to create a function that will remove one of the duplicated items from RDD such as (a,b) and (b,a)

In [ ]:
def sort_items(item):
    """Sorts the items by moving the tuple of strings from index 1 to index 0.
    Makes it possible to remove one of two duplicate pairs such as (a,b) and (b,a)."""
    
    if isinstance(item[1], tuple):
        return (item[1], item[0])
    
    else: 
        return item

Now we need to increase the RDD by doing the inner product between each unique item and the (PATIENT, (CODEs)) pairs.

In [ ]:
unique_items = full_log.distinct()

In [ ]:
combinedRdd = (groupedRdd.cartesian(unique_items)
            .filter(lambda item: item[0] != item[1])
            .filter(lambda item: len(item) == 2)
            .map(lambda item: sort_items(item))
            .distinct()
            )

In [15]:
combinedRdd2 = combinedRdd.map(lambda item: (item, 1))

In [16]:
freqRdd2 = (combinedRdd2.reduceByKey(lambda v1, v2: v1 + v2)
        .filter(lambda item: item[1] >= support_threshold)
        )

In [17]:
freqRdd2.top(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 26.0 failed 1 times, most recent failure: Lost task 15.0 in stage 26.0 (TID 255) (192.168.62.8 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\PySpark\spark-3.3.2-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 666, in main
  File "C:\PySpark\spark-3.3.2-bin-hadoop3\python\lib\pyspark.zip\pyspark\serializers.py", line 593, in read_int
    length = stream.read(4)
  File "C:\Users\franc\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
TimeoutError: timed out

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:552)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:758)
	at org.apache.s

In [9]:
supportRdd = supportRdd.reduceByKey(lambda x, y: x + y)
supportRdd.take(10)

[('37fac3c5-b7b5-4fc6-b6ff-2af0f610d203', 1),
 ('8d96ceb7-ba29-433c-93ca-a9fab7f4d4b9', 1),
 ('443165006', 225),
 ('32c0607e-916a-4ed3-b72f-b69c4d2a04c9', 1),
 ('f64d75c8-f608-49fb-a17b-2df7d893e6ba', 1),
 ('39847962-3d68-4e4d-b426-fee2ed332ae1', 1),
 ('d3b2fc65-883a-433b-ac41-348f52e9b78c', 1),
 ('5f52cd50-58f6-467e-b19f-3d5673a7a220', 1),
 ('446096008', 307),
 ('368581000119106', 279)]

Create a new RDD with the most frequent items, that exceed the support threshold (1000).

In [10]:
frequentRdd = supportRdd.filter(lambda item: item[1] >= support_threshold)

supportRdd = frequentRdd.map(lambda x: x[0]) # makes the support rdd only contain keys again

frequentRdd.take(10)

[('44465007', 1266),
 ('19169002', 2022),
 ('55822004', 1301),
 ('15777000', 3451),
 ('10509002', 5797),
 ('40055000', 2468),
 ('59621000', 3152),
 ('271737000', 3630),
 ('444814009', 12429),
 ('72892002', 4995)]

10 most frequent items of this first RDD.

In [11]:
frequentRdd.top(10, key = lambda x: x[1])

[('444814009', 12429),
 ('195662009', 6950),
 ('10509002', 5797),
 ('72892002', 4995),
 ('162864005', 3721),
 ('271737000', 3630),
 ('15777000', 3451),
 ('59621000', 3152),
 ('40055000', 2468),
 ('19169002', 2022)]

The first pass is now done, so next it will be counting the frequent items for k = 2.

First we create an RDD with all the combinations of the most frequent for k=1 with every unique value.

In [ ]:
combined = (supportRdd.cartesian(uniqueItems)
            .filter(lambda item: item[0] != item[1])
            .map(lambda item: (item, 1))
            )

combined.take(10)

In [ ]:
combinedSupport = combined.reduceByKey(lambda x, y: x[1] + y[1])
combinedSupport.take(10)

In [ ]:
combinedFrequent = combinedSupport.filter(lambda item: item[1] >= support_threshold)

combinedSupport = combinedFrequent.map(lambda x: x[0]) # makes the support rdd only contain keys again

combinedFrequent.take(10)

Function to remove replicas such as $(a,b)$ and $(b,a)$

In [ ]:
def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [ ]:
combined = combined.map(lambda item: removeReplica(item))
combined = combined.filter(lambda item: len(item) == 2)
combined.take(10)